In [65]:
import pandas as pd
import numpy as np

import torch
import msgpack
import os

from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer, CLIPTextModel

In [2]:
df = pd.read_csv('../output/fc_logprob_results.csv')
df['cum log prob'] = df['log probability'].cumsum()
df['cum prob'] = df['probability'].cumsum()
df.head(2)

,Unnamed: 0,index,total count,positive count,negative count,token size,phrase str,token_length,probability,log probability,frequency,pred log prob,cum log prob,cum prob
0,0,76,160021,2352,157669,2,blurry,1,0.000412,-7.794153,-0.128301,-14.890390,-7.794153,0.000412
1,1,57,147418,323,147095,2,bad anatomy,2,0.000057,-9.779522,-0.102254,-33.634502,-17.573676,0.000469


In [5]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").eval().to('cuda')

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.9.mlp.fc1.weight', 'vision_model.encoder.layers.21.self_attn.v_proj.weight', 'vision_model.encoder.layers.4.layer_norm2.weight', 'vision_model.encoder.layers.21.mlp.fc2.weight', 'vision_model.encoder.layers.20.self_attn.v_proj.weight', 'vision_model.encoder.layers.15.self_attn.k_proj.bias', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.1.layer_norm2.bias', 'vision_model.encoder.layers.2.self_attn.out_proj.weight', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.4.mlp.fc1.bias', 'vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encoder.layers.21.layer_norm2.bias', 'vision_model.encoder.layers.22.self_attn.k_proj.bias', 'vision_model.encoder.layers.7.layer_norm2.bias', 'vision_model.encoder.layers.5.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.layer_norm1.weig

In [63]:
def sampling_algo1():
    generated_prompt = []
    prompt_length = 0
    joint_prob = []
    joint_log_prob = []
    while True:
        # sample phrase
        random_number = np.random.rand(1)[0]
        row = df[df['cum prob'] >= random_number].iloc[0]
        phrase = row['phrase str']

        # check if prompt length exceeds 75 tokens
        tokens = tokenizer.tokenize(phrase + ', ')
        prompt_length += len(tokens)
        if prompt_length >= 75:
            break

        generated_prompt.append(phrase)
        joint_prob.append(row['probability'])
        joint_log_prob.append(row['log probability'])

    generated_prompt = ', '.join(generated_prompt)
    joint_prob = np.product(joint_prob)
    joint_log_prob = np.sum(joint_log_prob)

    with torch.no_grad():
        token_encoding = tokenizer(generated_prompt, return_length=True, return_tensors='pt')
        embedding = model(input_ids=token_encoding['input_ids'].to('cuda')).last_hidden_state[0]
    data = {
        'prompt': generated_prompt,
        'token_length': token_encoding['length'].item(),
        'joint_prob': joint_prob,
        'joint_log_prob': joint_log_prob,
        'embedding': embedding.cpu().numpy().tolist()
    }

    return data

In [64]:
save_path = '../output/prompt_algo1'


{'prompt': 'bright sky, evening, {{{slim figure:1.3}}}, 1girl, pussy, open cloth, depth_of_field, armchair, audience, jeans, red eyes, blurry, (ultra_detailed:1.3), high quality, narrow nose, hands between legs, detailed background, nature light, best quality',
 'token_length': 73,
 'joint_prob': 9.613731985309512e-76,
 'joint_log_prob': -172.73327456944025,
 'embedding': [[-0.3883765637874603,
   0.02294355258345604,
   -0.052196625620126724,
   -0.18406081199645996,
   -0.027318555861711502,
   -0.3355492949485779,
   -0.017579184845089912,
   -0.18692904710769653,
   0.18770648539066315,
   -0.09066228568553925,
   -0.22801178693771362,
   -0.14969751238822937,
   -0.07411383837461472,
   -0.3546871542930603,
   0.11344199627637863,
   -0.10165043920278549,
   -0.11977915465831757,
   -0.10019046813249588,
   0.04471952095627785,
   -28.09117317199707,
   -0.14018777012825012,
   -0.1786779910326004,
   -0.0848737359046936,
   -0.09500152617692947,
   -0.08985761553049088,
   -0.108